In [112]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
% matplotlib inline

In [113]:
depcols = ['review/appearance', 'review/aroma', 'review/overall',
           'review/palate', 'review/taste']

In [119]:
anna = pd.read_csv("anna.csv")[depcols + ["index", "stopwordcount", "charcount", "cursewordcount", "uniquecursewordcount"]]

df = pd.merge(pd.merge(pd.read_csv("tfidfsentiment.csv"), 
                       pd.read_csv("sentiments.csv"), 
                       on = "index"), 
              pd.read_csv("more_features.csv"), 
              on = "index")

df = pd.merge(df, anna, on = "index")

In [120]:
targets = df[depcols]
independents = df.drop(depcols, axis = 1)

In [121]:
independents.columns

Index(['exceptional_x', 'excellent_x', 'fantastic_x', 'wonderful_x',
       'highly_x', 'perfect', 'delicious', 'incredible', 'amazing', 'great',
       ...
       'sentiment_negative', 'sentiment_positive', 'sentiment_sadness',
       'sentiment_surprise', 'sentiment_trust', 'proportion_unique',
       'stopwordcount', 'charcount', 'cursewordcount', 'uniquecursewordcount'],
      dtype='object', length=1062)

In [122]:
independents.head()

,exceptional_x,excellent_x,fantastic_x,wonderful_x,highly_x,perfect,delicious,incredible,amazing,great,...,sentiment_negative,sentiment_positive,sentiment_sadness,sentiment_surprise,sentiment_trust,proportion_unique,stopwordcount,charcount,cursewordcount,uniquecursewordcount
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.263158,0.263158,0.052632,0.052632,0.105263,0.744681,42,415,0,0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.161290,0.193548,0.064516,0.064516,0.096774,0.660194,42,450,0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.421053,0.000000,0.052632,0.105263,0.850746,21,312,0,0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.300000,0.100000,0.000000,0.000000,0.100000,0.814286,27,312,0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.176471,0.176471,0.000000,0.176471,0.176471,0.747368,49,383,0,0


In [123]:
targets.head()

,review/appearance,review/aroma,review/overall,review/palate,review/taste
0,4.0,4.0,4.0,4.0,4.0
1,4.0,3.5,3.5,3.5,3.0
2,3.5,4.0,3.5,3.5,3.5
3,3.0,3.0,2.5,3.0,3.0
4,4.0,3.0,3.0,3.5,2.5


#### Apearance

In [141]:
np.random.seed(0)

validx = np.random.choice(df.index, size = int(df.shape[0] * .1), replace = False)
trainidx = [x for x in df.index if x not in validx]

In [142]:
dep = "review/appearance"

X_train = independents.iloc[trainidx].values
X_val = independents.iloc[validx].values

y_train = targets.iloc[trainidx].loc[:, dep].values
y_val = targets.iloc[validx].loc[:, dep].values

regr = linear_model.LinearRegression().fit(X_train, y_train)
print("Train R^2: " + str(regr.score(X_train, y_train)))
print("Val MSE: " + str(sum((regr.predict(X_val) - y_val) ** 2) / X_val.shape[0]))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Train R^2: 0.109839143203
Val MSE: 0.335278039416


#### Aroma

In [143]:
dep = "review/aroma"

X_train = independents.iloc[trainidx].values
X_val = independents.iloc[validx].values

y_train = targets.iloc[trainidx].loc[:, dep].values
y_val = targets.iloc[validx].loc[:, dep].values

regr = linear_model.LinearRegression().fit(X_train, y_train)
print("Train R^2: " + str(regr.score(X_train, y_train)))
print("Val MSE: " + str(sum((regr.predict(X_val) - y_val) ** 2) / X_val.shape[0]))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Train R^2: 0.129325493621
Val MSE: 0.44565626114


#### Palate

In [144]:
dep = "review/palate"

X_train = independents.iloc[trainidx].values
X_val = independents.iloc[validx].values

y_train = targets.iloc[trainidx].loc[:, dep].values
y_val = targets.iloc[validx].loc[:, dep].values

regr = linear_model.LinearRegression().fit(X_train, y_train)
print("Train R^2: " + str(regr.score(X_train, y_train)))
print("Val MSE: " + str(sum((regr.predict(X_val) - y_val) ** 2) / X_val.shape[0]))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Train R^2: 0.118284943022
Val MSE: 0.42990157284


#### Taste

In [145]:
dep = "review/taste"

X_train = independents.iloc[trainidx].values
X_val = independents.iloc[validx].values

y_train = targets.iloc[trainidx].loc[:, dep].values
y_val = targets.iloc[validx].loc[:, dep].values

regr = linear_model.LinearRegression().fit(X_train, y_train)
print("Train R^2: " + str(regr.score(X_train, y_train)))
print("Val MSE: " + str(sum((regr.predict(X_val) - y_val) ** 2) / X_val.shape[0]))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Train R^2: 0.130859991467
Val MSE: 0.480120483685


#### Overall

In [146]:
dep = "review/overall"

X_train = independents.iloc[trainidx].values
X_val = independents.iloc[validx].values

y_train = targets.iloc[trainidx].loc[:, dep].values
y_val = targets.iloc[validx].loc[:, dep].values

regr = linear_model.LinearRegression().fit(X_train, y_train)
print("Train R^2: " + str(regr.score(X_train, y_train)))
print("Val MSE: " + str(sum((regr.predict(X_val) - y_val) ** 2) / X_val.shape[0]))

# las_reg = linear_model.LassoCV(cv = 10).fit(X,y)
# print("Lasso R^2: " + str(las_reg.score(X, y)))
# print("Lasso SSE: " + str(sum((las_reg.predict(X) - y) ** 2)))

Train R^2: 0.11037856513
Val MSE: 0.472746881377
